In [57]:
import pandas as pd
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import warnings
warnings.simplefilter("ignore")

In [58]:
load_dotenv()

True

In [59]:
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [60]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

In [61]:
def get5minutedata(symbol, interval):
    frame = pd.DataFrame(session_auth.query_kline(symbol=symbol, interval=interval)["result"])
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [62]:
df = get5minutedata("ETHUSDT", "5m")
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-21 09:35:00,1649.73,1650.60,1649.49,1650.24,39.41416
2023-01-21 09:40:00,1650.24,1659.47,1650.18,1652.93,834.77356
2023-01-21 09:45:00,1652.93,1654.90,1649.12,1654.61,279.47472
2023-01-21 09:50:00,1654.61,1656.84,1653.66,1655.92,143.19987
2023-01-21 09:55:00,1655.92,1664.33,1655.90,1662.27,832.37187
...,...,...,...,...,...
2023-01-24 20:30:00,1616.98,1617.86,1616.04,1616.04,226.49404
2023-01-24 20:35:00,1616.04,1616.50,1612.73,1613.06,135.30520
2023-01-24 20:40:00,1613.06,1613.38,1608.42,1609.22,258.08453


In [66]:
eth = pd.read_csv ('./eth_actualizado.csv')
eth['Time'] = eth['time'].str.extract('(\d{4}-\d{2}-\d{2}\w\d{2}:\d{2}:\d{2})', expand=True)
eth['Time'] = pd.to_datetime(eth["Time"])
eth.set_index("Time", inplace = True)
eth.drop(columns="time", inplace=True)
eth.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
eth = eth[:7000]

In [67]:
eth

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-19 01:00:00,1182.50,1183.10,1182.45,1182.50,1871.65
2022-12-19 01:05:00,1182.50,1182.90,1181.35,1182.60,3669.31
2022-12-19 01:10:00,1182.60,1186.10,1182.60,1185.30,6792.08
2022-12-19 01:15:00,1185.30,1186.25,1184.35,1186.15,2287.35
2022-12-19 01:20:00,1186.15,1188.80,1186.15,1188.55,3974.38
...,...,...,...,...,...
2023-01-12 07:55:00,1398.55,1398.60,1397.70,1397.70,1819.98
2023-01-12 08:00:00,1397.70,1398.60,1397.30,1397.35,2349.89
2023-01-12 08:05:00,1397.35,1397.35,1396.15,1396.90,1162.84


In [109]:
class ETH_Short(Strategy):

    def init(self):
        close = self.data.Close
        self.rsi= self.I(ta.momentum.rsi, pd.Series(close), window=14)
        self.K = self.I(ta.momentum.stochrsi_k, pd.Series(close), window= 14)
        self.D = self.I(ta.momentum.stochrsi_d, pd.Series(close), window= 14)

    def next(self):
        price = self.data.Close
        rsi = self.rsi
        K = self.K
        D = self.D
       
        if  rsi > 68 and K + 0.03 < D and D > 0.75:
            sl = price * 1.01
            tp = price * 0.98
            self.sell(sl=sl, tp=tp)
    

In [110]:
bt = Backtest(df, ETH_Short, cash= 10000, commission= 0.0015)

In [111]:
output = bt.run()

In [112]:
output

Start                     2023-01-21 09:35:00
End                       2023-01-24 20:50:00
Duration                      3 days 11:15:00
Exposure Time [%]                        35.4
Equity Final [$]                  10550.52781
Equity Peak [$]                   10550.52781
Return [%]                           5.505278
Buy & Hold Return [%]               -2.134841
Return (Ann.) [%]                 2404.960633
Volatility (Ann.) [%]               291.08429
Sharpe Ratio                         8.262076
Sortino Ratio                             inf
Calmar Ratio                      1623.779039
Max. Drawdown [%]                   -1.481089
Avg. Drawdown [%]                   -0.340255
Max. Drawdown Duration        0 days 18:45:00
Avg. Drawdown Duration        0 days 02:36:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                       1.852779
Worst Trade [%]                      1.852779
Avg. Trade [%]                    

In [113]:
bt.plot()

Row(id='19938', ...)